<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/Pokemon_GPT_2_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon GPT-2 Train

This notebook will let you convert a folder of images to a text-based format and then train a GPT-2 model.

To use this notebook just follow along with the instructions and run each cell as directed.

*If you don't want to train a new model, but instead generate new Pokemon sprites with an existing model, you can skip to the generation notebook here: [Pokemon GPT-2 Generate](https://colab.research.google.com/drive/1qgt8cSwKF957PgPTKhRcNwDIfWrMhbV9)*

---

**Please Note**: Due to specifics of the text-based image format images can only be up to 99x99 pixels. If you want to change this, you'll need to modify the text conversion code to allow line numbers greater than 2 digits.

---

GPT-2 portions of this notebook are based on the notebook for [GPT-2-Simple](https://github.com/minimaxir/gpt-2-simple)

## 1. Setup

First we import everything we need and download the 124M GPT-2 model for retraining:

In [ ]:
import sys
import math
from PIL import Image
from os import listdir
import shutil

%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files, drive

gpt2.download_gpt2(model_name="124M")

## 2. Google Drive

Google Drive is used to load training images and store the resulting model. Here we mount your Google Drive:

In [ ]:
drive.mount('/content/drive')

## 3. Convert Images for Training

Now we'll convert your training images into one long text file that we'll use for training. By default we use a folder in your Google Drive titled "input-images". This folder can contain PNGs, GIFs, or JPGs. But as noted above, all images must be 99x99 pixels or smaller. My training used 64x64 sprites.

Please change this to the name of the folder in your Google Drive you want to convert, then run the cell:

In [ ]:
output = ''
input_folder = 'input-images' #@param {type:"string"}
input_folder_path = '/content/drive/My Drive/%s' % input_folder

for f in listdir(input_folder_path):
    ext = f.split('.')[-1]
    if ext not in ['png', 'gif', 'jpg', 'jpeg']:
      continue

    print('converting %s' % f)

    path = '%s/%s' % (input_folder_path, f)
    image = Image.open(path)
    pixels = image.load()
    width = image.size[0]
    height = image.size[1]

    lines = []

    for y in range(height):
        split = ['%02dd' % y]

        for x in range(width):
            color = pixels[x, y] 

            s = '~'

            if color[3] > 128:
                r = color[0]
                g = color[1]
                b = color[2]
               
                if r == 85 and g == 85 and b == 85:
                    s = 'a' # use grayscale character 1

                elif r == 170 and g == 170 and b == 170:
                    s = 'b' # use grayscale character 2

                else: # use one of 64 color characters
                    mR = math.floor(r/64)
                    mG = math.floor(g/64)
                    mB = math.floor(b/64)

                    c = 0;

                    c += mR
                    c = c << 2
                    c += mG
                    c = c << 2
                    c += mB

                    s = chr(c+33)

            split.append(s)

        lines.append(' '.join(split))

    reversed = []
    for line in lines:
        reversed.insert(0, (line.replace('d ', 'u ', 1)))

    output += '\n'.join(lines)
    output += '\n'.join(reversed)

print('saving output')
with open('all-images.txt', 'w') as output_file:
    output_file.write(output)
print('done !')

## 4. Train the Model

Now we will train a GPT-2 model on the converted images.

Set the checkpoint_folder to the name of the folder you want to save to in your Google Drive.

Also set run_name to a name that represents this training set.

The number of steps controls how long you will train your model on this data. Longer will probably lead to better results. 5000 is enough to see something. But 30000 or more is probably better. Try 5000, see what you get, and go from there.

Once you've set these values, run the cell:

In [ ]:
checkpoint_folder = 'pokemon-gpt-2-checkpoints'  #@param {type:"string"}
run_name = 'pokemon-gpt-2-run' #@param {type:"string"}
steps = 5000 #@param {type:"integer"}
session = gpt2.start_tf_sess()

session = gpt2.reset_session(session)
gpt2.finetune(session,
    dataset='all-images.txt',
    model_name='124M',
    steps=steps,
    restore_from='latest',
    run_name=run_name,
    print_every=10,
    sample_every=500,
    save_every=500
  )

shutil.copytree('checkpoint/%s' % run_name, '/content/drive/My Drive/%s/%s' % (checkpoint_folder, run_name))

## 5. Now Generate !

Once you've completed this notebook, head over to the [Pokemon GPT-2 Generate](https://colab.research.google.com/drive/1qgt8cSwKF957PgPTKhRcNwDIfWrMhbV9) notebook to generate sprites with your new model.